In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import copy
import cv2
from markovids import vid
import numpy as np
import h5py
import toml
from tqdm.auto import tqdm
import re

## User functions

In [3]:
def lp_filter(x, sigma):
    return cv2.GaussianBlur(x, [0, 0], sigma, sigma)

def bp_filter(x, sigma1, sigma2, clip=True):
    if (sigma1 == 0) or (sigma1 is None):
        return x
    elif (sigma2 == 0) or (sigma2 is None):
        return lp_filter(x, sigma1)
    else:
        return np.clip(
            lp_filter(x, sigma1) - lp_filter(x, sigma2),
            0 if clip == True else -np.inf,
            np.inf,
        )

# Load in metadata

In [4]:
base_dir = "/storage/home/hcoda1/4/jmarkowitz30/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/quantum_dots/timecourse_02_joints/"

# Gather all data files

In [5]:
import glob
dat_paths = {}
avis = sorted(glob.glob(os.path.join(base_dir, "**", "*reflectance.avi"), recursive=True))

# Get what already exists on segments

In [7]:
r = re.compile("session\_.*\d+.*\)")

In [9]:
exclude_paths = [
    "session_20240610170622-119479"
]
exclude_subjects = [
    "qd_knee_01",
    "qd_knee_02"
]

In [18]:
dry_run = False
force = False
weights = (0., 1.) # (fluo, reflectance)
# weights = None
max_hours = 72
bpass = None
batch_size = 2500
bground_path = "_bground"

In [19]:
# get some essentials
reader = vid.io.AutoReader(avis[0])

In [20]:
writer_kwargs = {"frame_size": reader.frame_size,
                 "dtype": reader.dtype,
                 "fps": reader.fps,
                 "pixel_format": reader.pixel_format}

In [21]:
reader.close()

In [25]:
# export_dir = os.path.join(f"/mnt/data/jmarkow/active_projects/keypoints_basler_nir_plexiglass_arena/keypoint_inference_export_fused_weights-{weights}_bpass-{bpass}/")
export_dir = os.path.join(f"/storage/home/hcoda1/4/jmarkowitz30/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/keypoints_basler_nir_plexiglass_arena/keypoint_inference_kneejoints_export_fused_weights-{weights}_bpass-{bpass}/")

In [26]:
os.path.exists(export_dir)

False

In [27]:
os.makedirs(export_dir, exist_ok=True)

In [28]:
import warnings

In [29]:
proc_avis = []
for _avi in tqdm(avis):
    
    skip = False
    
    for _exclude in exclude_paths:
        if _exclude in _avi:
            skip = True

        
    metadata = toml.load(os.path.join(os.path.dirname(_avi), "../metadata.toml"))
    
    user_metadata = metadata["user_input"]
    subject_name = user_metadata["subject"]
    session_name = user_metadata["session"]
    notes = user_metadata["notes"]

    if subject_name.lower() in exclude_subjects:
        skip = True

    if skip:
        warnings.warn(f"Skip path {_avi}")
        continue
    
    try:
        hours = int(re.match(r"\+(\d+)h", notes).groups(0)[0])
    except AttributeError:
        continue

    if (hours > max_hours):
        continue
    else:
        proc_avis.append(_avi)

  0%|          | 0/60 [00:00<?, ?it/s]

/tmp/ipykernel_3099460/1498407631.py:22: UserWarning: Skip path /storage/home/hcoda1/4/jmarkowitz30/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/quantum_dots/timecourse_02_joints/session_20240807184314-983785 (system76-pc)/_proc/Basler-267601741000-24383488-reflectance.avi
  warnings.warn(f"Skip path {_avi}")
/tmp/ipykernel_3099460/1498407631.py:22: UserWarning: Skip path /storage/home/hcoda1/4/jmarkowitz30/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/quantum_dots/timecourse_02_joints/session_20240807184314-983785 (system76-pc)/_proc/Basler-267601746219-24404505-reflectance.avi
  warnings.warn(f"Skip path {_avi}")
/tmp/ipykernel_3099460/1498407631.py:22: UserWarning: Skip path /storage/home/hcoda1/4/jmarkowitz30/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/quantum_dots/timecourse_02_joints/session_20240807184314-983785 (system76-pc)/_proc/Basler-267601746223-24404515-reflectance.avi
  warnings.warn(f"Skip path {_avi}")
/

In [30]:
proc_avis

['/storage/home/hcoda1/4/jmarkowitz30/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/quantum_dots/timecourse_02_joints/session_20240807185653-046788 (system76-pc)/_proc/Basler-267601741000-24383488-reflectance.avi',
 '/storage/home/hcoda1/4/jmarkowitz30/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/quantum_dots/timecourse_02_joints/session_20240807185653-046788 (system76-pc)/_proc/Basler-267601746219-24404505-reflectance.avi',
 '/storage/home/hcoda1/4/jmarkowitz30/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/quantum_dots/timecourse_02_joints/session_20240807185653-046788 (system76-pc)/_proc/Basler-267601746223-24404515-reflectance.avi',
 '/storage/home/hcoda1/4/jmarkowitz30/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/quantum_dots/timecourse_02_joints/session_20240807185653-046788 (system76-pc)/_proc/Basler-26760175421F-24461855-reflectance.avi',
 '/storage/home/hcoda1/4/jmarkowitz30/shared_folder/acti

In [31]:
for _avi in tqdm(proc_avis):
        
    dirname = os.path.dirname(_avi)
    fileparts = os.path.normpath(_avi).split(os.path.sep)
    fname = os.path.splitext(fileparts[-1])[0]
    session_date = list(filter(r.match, fileparts))[0]
    
    cam_name = os.path.splitext(os.path.basename(_avi))[0].split("-reflectance")[0] 
    metadata = toml.load(os.path.join(os.path.dirname(_avi), "../metadata.toml"))
    
    user_metadata = metadata["user_input"]
    subject_name = user_metadata["subject"]
    session_name = user_metadata["session"]
    notes = user_metadata["notes"]
    
    export_fname = f"{subject_name}_{session_name}-{notes}_{session_date}_camera-{cam_name}.avi"
    export_metadata_fname = f"{subject_name}_{session_name}-{notes}_{session_date}_camera-{cam_name}.toml"
    export_metadata = {"export_metadata": {"cam": cam_name, "session": session_name, "file": _avi}} 
    export_metadata["export_metadata"]["original_metadata"] = metadata
    # use_export_dir = os.path.join(export_dir, cam_name)
    save_path = os.path.join(export_dir, export_fname)
    
    
    os.makedirs(export_dir, exist_ok=True)
    writer_kwargs["pixel_format"] = "bgr24" if weights is None else "gray"
    
    if not dry_run:

        fluo_fname = _avi.replace("-reflectance.avi","-fluorescence.avi")
        reflect_reader = vid.io.AutoReader(_avi)
        fluo_reader = vid.io.AutoReader(fluo_fname)

        if os.path.exists(save_path) and not force:
            export_reader = vid.io.AutoReader(save_path)
            if reflect_reader.nframes == export_reader.nframes:
                # if we finished the job continue
                warnings.warn(f"Frames equal between {_avi} and {save_path}, skipping...")
                continue
            else: 
                warnings.warn(f"Frames unequal between {_avi} and {save_path}")
                pass

        writer = vid.io.AviWriter(save_path, **writer_kwargs)
        
    
        bground_fname = os.path.splitext(os.path.basename(fluo_fname))[0]
        use_bground_path = os.path.join(dirname, bground_path, f"{bground_fname}.hdf5")
        nframes = reflect_reader.nframes
        
        for _batch in tqdm(range(0, nframes, batch_size), total=int(np.ceil(nframes / batch_size))):
            
            use_frames = range(_batch, _batch + batch_size)
            reflect_frames = reflect_reader.get_frames(use_frames)

            # only load if we're using fluo
            if (weights is None) or (weights[1] != 1):
                fluo_frames = fluo_reader.get_frames(use_frames).copy()
                
                with h5py.File(use_bground_path, "r") as f:
                    rolling_bgrounds = f["bground"][()]
                    idxs = f["frame_idxs"][()]
            
                for i, (_idx, _frame) in enumerate(zip(use_frames, fluo_frames)):
                    use_bground = np.argmin(np.abs(idxs - _idx))
                    fluo_frames[i] = np.clip(_frame - rolling_bgrounds[use_bground], 0, 255).astype("uint8")
            else:
                # print("Skipping")
                fluo_frames = np.zeros_like(reflect_frames) # not used, just a dummy for iteration


            if weights is not None:
                write_frames = np.zeros_like(reflect_frames)
            else:
                write_frames = np.zeros(
                    (reflect_frames.shape[0],
                     reflect_frames.shape[1],
                     reflect_frames.shape[2],
                     3),
                    dtype="uint8")
            
            for i, (_reflect, _fluo) in enumerate(zip(reflect_frames, fluo_frames)):    
                if bpass is not None:
                    proc_fluo = bp_filter(_fluo.astype("float32"), *bpass)
                else:
                    proc_fluo = _fluo.astype("float32")
                proc_fluo[proc_fluo<=0] = 0

                if weights is not None:
                    if weights[1] == 1:
                        new_im = _reflect.astype("uint8") # skip normalization if we're not combining
                    elif weights[0] == 1:
                        new_im = proc_fluo.astype("uint8") # skip normalization if we're not combining
                    else:
                        new_im = weights[0] * proc_fluo + weights[1] * _reflect
                        new_im = (new_im - new_im.min()) / (new_im.max() - new_im.min())
                        new_im *= 255
                        new_im = new_im.astype("uint8")
                    write_frames[i] = new_im
                else:
                    write_frames[i][...,0] = proc_fluo.astype("uint8")
                    write_frames[i][...,1] = _reflect.astype("uint8")
                    write_frames[i][...,2] = _reflect.astype("uint8")
                    
            writer.write_frames(write_frames, progress_bar=False)
        
        reflect_reader.close()
        fluo_reader.close()
        writer.close()
        
        with open(os.path.join(export_dir, export_metadata_fname), "w") as f:
            toml.dump(export_metadata, f)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]